In [1]:
import numpy as np
import tensorflow as tf
import gym
#loading mnist
env = gym.make('CartPole-v0')

In [ ]:
H=50
batch_size=25
learning_rate=1e-1
D=4
gamma=0.99

def discount_rewards(r):
    discounted_r = np.zeros_like(r)
    running_add = 0
    for t in reversed(range(r.size)):
        running_add = running_add * gamma + r[t]
        discounted_r[t] = running_add
    return discounted_r

observations = tf.placeholder(tf.float32, [None,D], name="input_x")

W1 = tf.get_variable("W1",shape=[D,H],initializer=tf.contrib.layers.xavier_initializer())

layer1 = tf.nn.relu(tf.matmul(observations,W1))

W2 = tf.get_variable("W2",shape=[H,1],initializer=tf.contrib.layers.xavier_initializer())

score = tf.matmul(layer1,W2)

probability = tf.nn.sigmoid(score)


input_y = tf.placeholder(tf.float32,[None,1],name="input_y")

advantages = tf.placeholder(tf.float32,name="reward_signal")

loglik = tf.log(input_y*(input_y - probability) + \
                (1 - input_y)*(input_y + probability))

#loglik * advantages(rewards) !!cretical process in reinforcement learning
loss =- tf.reduce_mean(loglik*advantages)

tvars = tf.trainable_variables()

newGrads = tf.gradients(loss,tvars)

Adadelta = tf.train.AdadeltaOptimizer(learning_rate=learning_rate)

W1Grad = tf.placeholder(tf.float32,name="batch_grad1")

W2Grad = tf.placeholder(tf.float32,name="batch_grad2")

batchGrad = [W1Grad,W2Grad]

updateGrads = Adadelta.apply_gradients(zip(batchGrad,tvars))

xs,ys,drs = [],[],[]

reward_sum = 0

episode_number = 1

total_episodes = 10000

with tf.Session() as sess:
    rendering = False
    init = tf.global_variables_initializer()
    sess.run(init)
    observation = env.reset()
    
    gradBuffer = sess.run(tvars)
    for ix,grad in enumerate(gradBuffer):
        gradBuffer[ix] = grad*0
    
    while episode_number <= total_episodes:
        #if reward exceeded 100, display performance
        if reward_sum/batch_size > 100 or rendering == True:
            env.render()
            rendering = True
            
        x = np.reshape(observation,[1,D])
        
        #count the number which determines whether action = 1 or 0
        tfprob = sess.run(probability,feed_dict = {observations:x})
        action = 1 if np.random.uniform() < tfprob else 0
        
        xs.append(x)
        y = 1 - action
        ys.append(y)
        
        obervation, reward, done, info = env.step(action)
        reward_sum += reward
        
        drs.append(reward)
        
        if done:
            episode_number +=1
            #observation stack
            epx = np.vstack(xs)
            #label stack
            epy = np.vstack(ys)
            #reward stack
            epr = np.vstack(drs)
            xs,ys,drs = [],[],[]
            
            #stabilizing rewards
            discounted_epr = discount_rewards(epr)
            discounted_epr -= np.mean(discounted_epr)
            discounted_epr /= np.std(discounted_epr)
            
            #counting gradient
            tGrad = sess.run(newGrads, feed_dict={observations: epx, input_y: epy, advantages: discounted_epr})
            
            for ix,grad in enumerate(tGrad):
                gradBuffer[ix] += grad
            
            if episode_number % batch_size ==0:
                sess.run(updateGrads,feed_dict = {W1Grad: gradBuffer[0], W2Grad: gradBuffer[1]})
                
                for ix,grad in enumerate(gradBuffer):
                    gradBuffer[ix] = grad*0
                print('Average reward for episode %d : %f.' % \
                      (episode_number, reward_sum/batch_size))
                
                if reward_sum/batch_size > 200:
                    print("Task solved in", episode_number, 'episodes!')
                    break
                    
                reward_sum=0
            
            obervation = env.reset()

W0826 15:49:27.772167 140066457925440 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



Average reward for episode 25 : 17.960000.
Average reward for episode 50 : 23.400000.
Average reward for episode 75 : 17.560000.
Average reward for episode 100 : 23.240000.
Average reward for episode 125 : 23.920000.
Average reward for episode 150 : 23.920000.
Average reward for episode 175 : 21.520000.
Average reward for episode 200 : 24.760000.
Average reward for episode 225 : 19.560000.
Average reward for episode 250 : 19.720000.
Average reward for episode 275 : 19.000000.
Average reward for episode 300 : 19.720000.
Average reward for episode 325 : 19.600000.
Average reward for episode 350 : 19.760000.
Average reward for episode 375 : 21.360000.
Average reward for episode 400 : 22.880000.
Average reward for episode 425 : 22.880000.
Average reward for episode 450 : 22.920000.
Average reward for episode 475 : 22.000000.
Average reward for episode 500 : 26.160000.
Average reward for episode 525 : 23.320000.
Average reward for episode 550 : 22.560000.
Average reward for episode 575 : 22